# Semantic functions

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"

#!import Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    .Build();

### ✍️ Create a simple plain semantic function

In [ ]:
var promptTemplate = "When is the next total solar eclipse?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);
var response = await kernel.InvokeAsync(semanticFunction);

display(response.GetValue<string>())

### 📋 Now let's create a more complete semantic function (with input variable and execution settings)

In [ ]:
var promptTemplate = "When is the next total {{$input}} eclipse? Respond only with JSON.";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 500,
    Temperature = 0.1,
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate, executionSettings);
var response = await kernel.InvokeAsync(semanticFunction, new() { ["input"] = "lunar" });

display(response.GetValue<string>())

### 📋 But what about if we have more input variables?
### 🪴 This simple bit of "grounding" reduces so-called 'hallucination' with LLMs

In [ ]:
var promptTemplate = "When is the next total {{$input}} eclipse? Today is {{$today}}. Respond only with JSON.";

var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 500,
     Temperature = 0.1 
};
var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "solar",
    ["today"] = "22st of April 2024"
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

### In this example, the plugin we've added `DateTimePlugin` is used in the templated prompt. The native function GetCurrentDate is invoked to enrich the prompt context with current date.

In [ ]:
var promptTemplate = "Considering that today date is {{date.GetCurrentDate}}, how many days until {{$input}}?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())